# MATH 610 Programming Assignment 3
We consider the following problem: find $u : \Omega \times (0,T) \to \mathbb R$ such that
$$ \partial_t u - \Delta u + q u = f $$ on $\Omega \times (0,T)$ with initial condition $u(x,0) = 0$ for $x \in \Omega$ and boundary condition $n \cdot \nabla u = g$ on $\partial \Omega \times (0,T)$.

## Weak formulation
We multiply by a test function $v : \Omega \to \mathbb R$  and integrate-by-parts to arrive at the following weak formulation at each $t \in (0,T)$:
$$ \frac{d}{dt}(u(\cdot,t),v)_{\Omega} + (\nabla u, \nabla v)_\Omega + q(u,v)_\Omega = (f(\cdot,t),v)_\Omega + (g(\cdot,t),v)_{\partial\Omega}.$$
Here, we let $(\cdot,\cdot)_\Omega$ denote the $L^2$ inner product in space on $\Omega$, and we make a similar definition for $(\cdot,\cdot)_{\partial\Omega}$.

## Discretizing in space
Now we let $\varphi_i : \Omega \to \mathbb R$ denote our finite element basis functions and we assume that we can write $u$ as a linear combination
$$ u(x,t) = \sum_j u_j(t)\varphi_j(x) $$
where the $u_j : (0,T) \to \mathbb R$ are the coefficients at time $t$ with respect to the basis $\varphi_i$.
Inserting this into our weak formulation and testing with $v = \varphi_i$ gives us the following system of ODEs
$$ MU'(t) + SU(t) + qMU(t) = F(t) + G(t) $$
where
$$ U_j(t) = u_j(t) $$
$$ M_{ij} = (\varphi_j,\varphi_i)_\Omega $$
$$ S_{ij} = (\nabla\varphi_j, \nabla\varphi_i)_\Omega $$
$$ F_i(t) = (f(\cdot,t),\varphi_i)_\Omega $$
and
$$ G_i(t) = (g(\cdot,t), \varphi_i)_{\partial\Omega}. $$

## Discretizing in time
We can discretize this system of ODEs using either a forward Euler discretization in time:
$$ M \frac{U^{n+1}-U^n}{k} + SU^n + q M U^n = F^n + G^n, $$
or by using a backward Euler discretization:
$$ M \frac{U^{n+1}-U^n}{k} + SU^{n+1} + q M U^{n+1} = F^{n+1} + G^{n+1}. $$

Then, starting with initial condition $U^0 = 0$, we can iteratively solve for $U^{n+1}$ from $U^n$ by algebraically solving the matrix-vector equations above.
Explicitly, with forward Euler:
$$ MU^{n+1} = ((1-kq)M - kS)U^n + k(F^n + G^n) $$
and for backward Euler:
$$ ((1+kq)M + kS)U^{n+1} = MU^n + k(F^{n+1} + G^{n+1})$$

In [ ]:
import meshpy.triangle
import scipy.sparse
import scipy.sparse.linalg
import numpy as np
import matplotlib.pyplot as plt

print("Running")

# Parameters
max_cell_area = 1 / 10**2
final_time = 1
n_time_steps = 20
time_step_size = final_time / n_time_steps

vertices = [[0, 0], [1, 0], [1, 1], [0, 1]]
edges = [[0, 1], [1, 2], [2, 3], [3, 0]]
holes = [[0.5,0.5]]

q_coefficient = 5
plot_solution = True
have_exact_solution = True
compute_errors = True
plot_filename = "solution.png"


def exact_solution(x, t):
    return t * np.exp(-t) * np.cos(3 * np.pi * x[0]) * np.cos(np.pi * x[1])


def grad_exact_solution(x, t):
    return (
        -t
        * np.exp(-t)
        * np.pi
        * np.array(
            [
                3 * np.sin(3 * np.pi * x[0]) * np.cos(np.pi * x[1]),
                np.cos(3 * np.pi * x[0]) * np.sin(np.pi * x[1]),
            ]
        )
    )


def laplace_exact_solution(x, t):
    return -10 * np.pi**2 * exact_solution(x, t)


def d_exact_solution_dt(x, t):
    return (1 - t) * np.exp(-t) * np.cos(3 * np.pi * x[0]) * np.cos(np.pi * x[1])


def f(x, t):
    return (
        d_exact_solution_dt(x, t)
        - laplace_exact_solution(x, t)
        + q_coefficient * exact_solution(x, t)
    )


def g(x, t):
    return 0


# Make a mesh
mesh = meshpy.triangle.MeshInfo()
mesh.set_points(vertices)
mesh.set_facets(edges)
mesh.set_holes(holes)
mesh = meshpy.triangle.build(mesh, max_volume=max_cell_area)

# Get mesh info
cells = mesh.elements
vertices = mesh.points
boundary_edges = mesh.facets
n_dofs = len(vertices)

# Compute the mesh size
mesh_size = 0.0
v0 = np.zeros(2)
v1 = np.zeros(2)
v2 = np.zeros(2)
for c in range(len(cells)):
    v0[:] = vertices[cells[c][0]]
    v1[:] = vertices[cells[c][1]]
    v2[:] = vertices[cells[c][2]]
    mesh_size = max(
        mesh_size,
        max(np.linalg.norm(v1 - v0), np.linalg.norm(v2 - v0), np.linalg.norm(v2 - v1)),
    )

# Initialize matrices and vectors
mass_matrix = scipy.sparse.lil_array((n_dofs, n_dofs))
stiffness_matrix = scipy.sparse.lil_array((n_dofs, n_dofs))
cell_rhs = np.zeros(n_dofs)
boundary_rhs = np.zeros(n_dofs)
initial_condition = np.zeros(n_dofs)
solution = np.zeros(n_dofs)

# Cell quadrature info
cell_quad_pts = [
    np.array(point)
    for point in [
        (0.33333333333333333, 0.33333333333333333),
        (0.79742698535308720, 0.10128650732345633),
        (0.10128650732345633, 0.79742698535308720),
        (0.10128650732345633, 0.10128650732345633),
        (0.05971587178976981, 0.47014206410511505),
        (0.47014206410511505, 0.05971587178976981),
        (0.47014206410511505, 0.47014206410511505),
    ]
]
cell_quad_wts = [
    w / 2
    for w in [
        0.22500000000000000,
        0.12593918054482717,
        0.12593918054482717,
        0.12593918054482717,
        0.13239415278850616,
        0.13239415278850616,
        0.13239415278850616,
    ]
]
n_cell_quad_pts = len(cell_quad_pts)

# Cell basis info
cell_bases = [lambda x: 1 - x[0] - x[1], lambda x: x[0], lambda x: x[1]]
cell_gradients = [np.array([-1, -1]), np.array([1, 0]), np.array([0, 1])]

# Tabulate basis functions at the quadrature points
cell_basis_values_at_quad_pts = [
    [cell_bases[i](cell_quad_pts[q]) for i in range(len(cell_bases))]
    for q in range(n_cell_quad_pts)
]

# Cell mapping info
v0 = np.zeros(2)
J = np.zeros((2, 2))
invJ = np.zeros((2, 2))

# Assemble system matrix and mass matrix since they are time-independent
print("Assembling system matrix and mass matrix")
grad_basis_iq = np.zeros(2)
grad_basis_jq = np.zeros(2)
for c in range(len(cells)):
    print("Cell", c, "out of", len(cells))
    # Update mapping information for cell
    v0[:] = vertices[cells[c][0]]
    J[:, 0] = vertices[cells[c][1]] - v0
    J[:, 1] = vertices[cells[c][2]] - v0
    invJ[:, :] = np.linalg.inv(J)
    detJ = abs(np.linalg.det(J))

    # Do quadrature on the cell
    for q in range(n_cell_quad_pts):
        weight = cell_quad_wts[q]
        mapped_weight = detJ * weight
        for i in range(3):
            row = cells[c][i]
            basis_iq = 0.0  # FIXME
            grad_basis_iq[:] = 0.0  # FIXME
            for j in range(3):
                col = cells[c][j]
                basis_jq = 0.0  # FIXME
                grad_basis_jq[:] = 0.0 # FIXME

                mass_matrix[row, col] += 0.0  # FIXME
                stiffness_matrix[row, col] += 0.0  # FIXME

# Edge quadrature info
edge_quad_pts = [
    0.5 + 0.5 * point for point in [np.sqrt(1.0 / 3.0), -np.sqrt(1.0 / 3.0)]
]
edge_quad_wts = [0.5, 0.5]
n_edge_quad_pts = len(edge_quad_pts)

# Edge basis info
edge_bases = [lambda x: 1 - x, lambda x: x]

# Tabulate basis functions at the quadrature points
edge_basis_values_at_quad_pts = [
    [edge_bases[i](edge_quad_pts[q]) for i in range(len(edge_bases))]
    for q in range(n_edge_quad_pts)
]
print("Assembled")

# Backward Euler time stepping
print("Solving with backward Euler timestepping")
backward_euler_rhs = np.zeros(n_dofs)

backward_euler_matrix = 0.0  # FIXME

cell_quad_pt = np.zeros(2)
v1 = np.zeros(2)
for i in range(n_time_steps):
    print("Timestep", i, "out of", n_time_steps)
    # Update from previous time step
    initial_condition[:] = solution[:]
    cell_rhs[:] = 0.0
    boundary_rhs[:] = 0.0
    t = (i + 1) * time_step_size

    # Assemble cell rhs
    for c in range(len(cells)):
        # Update mapping information for cell
        v0[:] = vertices[cells[c][0]]
        J[:, 0] = vertices[cells[c][1]] - v0
        J[:, 1] = vertices[cells[c][2]] - v0
        detJ = abs(np.linalg.det(J))

        # Do quadrature on the cell
        for q in range(n_cell_quad_pts):
            weight = cell_quad_wts[q]
            mapped_weight = detJ * weight
            cell_quad_pt[:] = cell_quad_pts[q]
            f_q = f(v0 + np.dot(J, cell_quad_pt), t)
            for i in range(3):
                row = cells[c][i]
                basis_iq = 0.0  # FIXME
                cell_rhs[row] += 0.0  # FIXME

    # Assemble boundary rhs
    for e in range(len(boundary_edges)):
        # Update mapping information for edge
        v0[:] = vertices[boundary_edges[e][0]]
        v1[:] = vertices[boundary_edges[e][1]]
        len_e = np.linalg.norm(v1 - v0)

        # Do quadrature on the edge
        for q in range(n_edge_quad_pts):
            weight = edge_quad_wts[q]
            mapped_weight = len_e * weight
            edge_quad_pt = edge_quad_pts[q]
            g_q = g(v0 * (1 - edge_quad_pt) + v1 * edge_quad_pt, t)
            for i in range(2):
                row = boundary_edges[e][i]
                basis_iq = 0.0  # FIXME
                boundary_rhs[row] += 0.0  # FIXME

    # Solve backward Euler system
    backward_euler_rhs[:] = 0.0  # FIXME
    solution[:] = scipy.sparse.linalg.spsolve(backward_euler_matrix, backward_euler_rhs)

# Plot solution at final time
if plot_solution:
    print("Finished")
    print("Plotting solution")
    fig = plt.figure()
    xs = [v[0] for v in vertices]
    ys = [v[1] for v in vertices]
    if have_exact_solution:
        fig.suptitle("Approximate (left) vs Exact (right)")
        ax = fig.add_subplot(1, 2, 1, projection="3d")
        ax.plot_trisurf(xs, ys, solution, triangles=cells, cmap="viridis")
        ax = fig.add_subplot(1, 2, 2, projection="3d")
        ax.plot_trisurf(
            xs,
            ys,
            [exact_solution(v, final_time) for v in vertices],
            triangles=cells,
            cmap="viridis",
        )
    else:
        fig.suptitle("Approximate")
        ax = fig.add_subplot(1, 1, 1, projection="3d")
        ax.plot_trisurf(xs, ys, solution, triangles=cells, cmap="viridis")
    plt.savefig(plot_filename)
    print("Solution saved to", plot_filename)

# Compute L2 error and H1 error at final time
if have_exact_solution and compute_errors:
    print("Computing errors")
    cell_quad_pt = np.zeros(2)
    grad_exact_q = np.zeros(2)
    grad_solution_on_cell = np.zeros(2)
    L2_error_sq = 0.0
    H1_error_sq = 0.0
    for c in range(len(cells)):
        # Update mapping information for cell
        v0[:] = vertices[cells[c][0]]
        J[:, 0] = vertices[cells[c][1]] - v0
        J[:, 1] = vertices[cells[c][2]] - v0
        invJ[:, :] = np.linalg.inv(J)
        detJ = abs(np.linalg.det(J))

        # Do quadrature on the cell
        for q in range(n_cell_quad_pts):
            weight = cell_quad_wts[q]
            cell_quad_pt[:] = v0 + J.dot(cell_quad_pts[q])
            exact_q = exact_solution(cell_quad_pt, final_time)
            grad_exact_q[:] = grad_exact_solution(cell_quad_pt, final_time)
            mapped_weight = detJ * weight
            solution_value_on_cell = 0.0
            grad_solution_on_cell[:] = 0.0
            for i in range(3):
                row = cells[c][i]
                basis_iq = 0.0  # FIXME
                grad_basis_iq[:] = 0.0  # FIXME
                solution_value_on_cell += solution[row] * basis_iq
                grad_solution_on_cell[:] += solution[row] * grad_basis_iq
            cell_L2_error_sq = (exact_q - solution_value_on_cell) ** 2 * mapped_weight
            L2_error_sq += cell_L2_error_sq
            H1_error_sq += (
                cell_L2_error_sq
                + np.linalg.norm(grad_exact_q - grad_solution_on_cell) ** 2
                * mapped_weight
            )
    L2_error = np.sqrt(L2_error_sq)
    H1_error = np.sqrt(H1_error_sq)
    print(
        "problem",
        "n_timesteps",
        "final_time",
        "timestep_size",
        "n_dofs",
        "mesh_size",
        "L2_error",
        "H1_error",
    )
    print(
        1,
        n_time_steps,
        final_time,
        time_step_size,
        n_dofs,
        mesh_size,
        L2_error,
        H1_error,
    )
print("Done")